#  9. Solidity

* Last Updated 20190605WEB1200 20190120SUN0000 20180707SAT1300 20170112
* todo: 1) ```watch``` 2) library ```myLib+StringUtils``` 3) 계정에서 발생한 거래의 목록? 4) registry

## 9.1 학습내용

### 9.1.1 목표
 
* solidity 파일
* value type, reference type
* bool, int, string, address 등 데이터타잎을 배우기

### 9.1.2 목차

* 9.2 [Solidity File](#9.2)
    * Pragma
    * Import
    * Contract
    * comments
* 9.3 컨트랙의 구조
    * SimpleContract.sol
* 9.4 데이터 타잎
    * 9.4.1 타잎 구분
    * 실습: DataTypeTest.sol
    * 9.4.2 [계정주소](#9.4.2)
    * 9.4.3 [저장장소](#9.4.3)
    * 9.4.4 [가시성](#9.4.3)
    * 실습: 주소 계정에서 입출금 myBank.sol
    * 9.4.5 [conversion](#9.4.5)
    * 실습: DataConversionTest.sol
    * 9.4.6 [전역 변수](#9.4.6)
    * 실습: GlobalVarsTest.sol
    * 살습: 은행송금 BankV2

## 9.2 Solidity File
<a id='9.2'></a>

Solidity File은 contract 소스코드를 확장자 ```.sol```로 저장한 파일이다.
파일에는 버전 pragma, 다른 소스코드를 불러오는 import문, 도움말 comments를 포함할 수 있다.

### Pragma
```pragma```는 컴파일러 버전을 선택할 때 사용한다.
반드시 적어주지 않아도 되지만 프로그램의 첫 줄에 적는다.

```python
pragma solidity <버전>;
```

### Import
import는 filename의 프로그램을 읽어들여 사용하게 된다.

```python
import "filename"
```
그 파일 내 **컨트랙, 전역변수** 등을 그대로 가져오기 때문에 주의를 해야 한다.
이로 인해 현재 파일의 전역변수와 문제를 일으킬 수 있으므로
```import ... as ...``` 구문을 사용하기도 한다.

```python
import "filename" as symbolName
```

### Contract

Contract은 비유를 하면, 객체지향 클래스에 해당한다. 곧 상세하게 설명하게 된다.
Contract은 블록체인 상의 프로그램이지만, 그 개념이나 구현하는 객체지향언어에서 사용하는 **클래스와 유사**하다.
프로그램 단위로서 Contract으로서 필요한 기능을 제공하는다.
따라서 클래스와 같이, Contract은 생성자, 속성, 함수로 구성한다.
물론 객체지향의 상속, 다형성을 사용할 수 있다.
따라서 프로그래밍은 Contract을 만들어 나가면서 이루어지게 된다.
CapsWord 스타일로 단어의 첫글자는 대문자로 적어준다




### Comments
<a id='S.1.4'></a>

도움말을 적는다. C 또는 자바 언어 스타일과 동일하다.
한 줄은 ```//```라고 맨 앞에 적어주어 도움말로 만든다.
여러 줄은 해당 블록 앞 뒤에 ```/*...*/```를 적어준다.
Doxygen 스타일의 @태그를 사용하는 것도 가능하다.
```python
/**
@param p1 The first parameter
@param p2 The second parameter
@returns the returned results
*/
```

## 9.3 컨트랙의 구조

컨트랙은 객체지향의 클래스와 같이 구성하면 된다.
클래스가 (1) 멤버 **속성**과 (2) 멤버 **함수**로 구성하듯이 컨트랙도 마찬가지이다.
객체지향으로서 상속을 사용하여 확장을 할 수도 있다.

다음 ```SimpleContract.sol```은 단순한 컨트랙의 예제이다.
* 컨트랙의 멤버 속성 **state variables**으로 ```address```, ```mapping```을 정의
* ```event PrintLog``` 이벤트를 선언.
* 함수 function ```constructor```, ```deposit()```, ```withdraw()```, ```queryBalance()``` 또는 modifier를 가질 수 있다.


In [1]:
%%writefile src/SimpleContract.sol
pragma solidity ^0.5.0;
contract SimpleContract {
    address public owner;
    mapping (address => uint) private balances;

    event PrintLog(address addr, uint amount);

    constructor() public {
        owner = msg.sender;
    }
    function deposit(uint amount) public payable returns (uint) {
        require(amount==msg.value);
        balances[msg.sender] += msg.value;
        emit PrintLog(msg.sender, msg.value);
        return balances[msg.sender];
    }

    function withdraw(uint amount) public returns (uint) {
        if (amount <= balances[msg.sender]) {
            balances[msg.sender] -= amount;
            msg.sender.transfer(amount);
        }
        return balances[msg.sender];
    }
    function queryBalance() public view returns (uint) {
        return balances[msg.sender];
    }
}

Writing src/SimpleContract.sol


## 9.4 데이터 타잎
<a id='9.3'></a>

Solidity는 다른 언어에서 지원하는 데이터타잎을 큰 차이 없이 사용할 수 있지만, 몇 가지 주의할 점이 있다.
블럭체인은 그 특성상 저장공간을 많이 사용할수록 비용이 발생하기 때문에 **저장공간을 효율적**으로 사용해야 한다.
그리고 소수점은 아직 지원되지 않아 float, double과 같은 자료형이 없다.

### 9.4.1 타잎 구분

데이터 타잎은 (1) value type, (2) reference type으로 크게 구분할 수 있다.

#### **Value Type**

데이터 그 자체의 **값**으로 저장되고, 전달되는 타잎이다.
다른 변수에 할당되거나 함수인자로 전달될 경우, 값이 복사가 되어 전달되는 타잎이다.
```bool```, ```int/uint```, ```address```, ```bytes (1~32 바이트)```, ```enum```이 해당된다.
value type은 **32바이트**로 그 값을 저장할 수 있다.

#### **참조 타잎 Reference Type**

**32바이트를 넘는 크기의 데이터**는 그 저장주소를 저장한다.
그 저장주소가 복사되어 전달되기 때문에 원본 데이터가 변경되면 따라서 변경된다.
```bytes```, ```Array```, ```String```, ```Struct```, ```mapping```이 해당된다.

데이터타잎 | 설명
----------|----------
bool | uint8, 0 and 1의 값을 가진다
uint<M> | unsigned integer, M은 8비트 단위로 256바이트까지 가능하다 0 < M <= 256, M % 8 == 0
int<M> | signed integer, M은 8비트 단위로 256비트까지 가능하다 0 < M <= 256, M % 8 == 0
address | 주소. 크기가 20바이트이므로 uint160 이다.
uint, int | uint256, int256
bytes<M> | 바이너리 타잎, M은 32바이트까지 가능하다 0 < M <= 32. **```bytes``` (크기가 없는)는 value type이 아니다**.
function | bytes24, 주소 + function selector
string | UTF-8 문자열, **value type이 아니다**.

#### boolean
Boolean은 true, false 둘 중의 하나 값을 가지는 데이터타잎이다.
비교연산자 (==, !=, !, >, <), 논리연산자 (&& ||)의 결과가 true, false 와 같이 boolean이 된다.

```python
bool isMarried=true;
```

#### int
```int```는 정수를 가지는 타잎이고 ```signed```, ```unsigned```로 구분할 수 있다.
* ```signed int```는 양수, 음수 모두 허용되며
* ```unsigned int```는 양수만 사용할 수 있다.

그 크기는 8, 16, 24, 32 이런 식으로 8바이트 단위로 증가하고 **최대 32**바이트까지 사용할 수 있다.
크기를 적지 않으면 256비트로 간주한다.
```+, -, *, /, %, **, ++, --, +=, -=```와 같은 산술연산자를 사용할 수 있다.

```python
int256 x=1;
```


#### enumeration

enum은 요소로 구성된 데이터 타잎을 정의할 경우 사용한다.
예를 들면, 요일과 같이 월, 화, 수, 목, 금, 토를 '집합'을 구성할 경우 enum을 사용한다. 
요소는 0부터 시작하여 정수 값을 가진다.
**비교**에 사용하며, 존재하지 않는 선택으로 인한 오류를 막을 수 있어 유용하다.
아래는 성별을 남, 녀로 구성한 enum이다.
**enum이름.변수명**으로 사용한다.

```python
enum gender {male, female}
enum Day {SUNDAY, MONDAY, TUESDAY, WEDNESDAY, THURSDAY, FRIDAY, SATURDAY}
```

#### 고정크기 byte array

```bytes```는 ```bytes1``` ~ ```bytes32``` 사이의 어떤 크기로 선언할 수 있다.
그러니까 **최대 32바이트까지 1바이트 단위**로 크기를 늘릴 수 있다.
byte, bytes32는 hex 값 뿐만 아니라 정수, 문자, 문자열을 가질 수 있다.
```byte```는 1 바이트, 8비트의 길이를 가진다. 따라서 ```byte```는 ```bytes1```과 동일하다. 

```python
bytes1 x= 0xFF;
bytes23 place1 = "7 hongji-dong jongro-gu";
bytes8 place2 = "7 hongji"; //결과값은 0x3720686f6e676a69
```

문자 | ascii | hex
-----|-----|-----
7 | 55 | 37
space | 32 | 20
h | 104 | 68
o | 111 | 6f
n | 110 | 6e
g | 103 | 67
j | 106 | 6a
i | 105 | 69

#### 변동크기 byte array

```bytes``` (바이크 크기를 의미하는 숫자가 없는) 또는 ```string```은 그 크기가 정해져 있지 않아서 가변적이다.
길이 제한이 없는 경우 사용한다.
string은 UTF8 형식으로 문자열을 저장한다.

크기를 알려주는 bytes(s).length and bytes(s)[2]를 사용할 수 있다.

```python
string n = "hello";
```

#### struct

struct은 서로 관련있는 데이터를 그에 맞는 데이터타잎으로 구성하여 묶어서 사용할 수 있다. CapWords 스타일로 첫글자는 대문자로 적어준다.

```python
struct Student {
        uint num;
        string name;
        bool isEnrolled; 
}
student s1=Student(201911111,"jslim",true); 
```

* constant 상수
```python
string constant name="jsl";
```

다음 명령문 ```^0.5.0```은 메이저버전 5, 마이너버전 0을 의미한다.
맨 앞 ```^``` caret는 메이저버전으로 시작하는 최신버전을 선택한다는 의미이다.
따라서 현재 발표된 최신 버전이 0.5.3이므로, 이를 선택하게 된다.

줄 | 설명
-----|-----
1 | 컴파일러 0.5의 최신 버전으로 설정
5 | ```bytes1```은 ```byte```과 동일한 의미
6 | ```bytes23```에는 23글자를 입력. 한 글자를 추가하면 오류가 발생된다.
7 | ```bytes8```에는 8글자를 입력.
17 | ```bytes1```로 선언된 b1을 ```byte```로 반환한다. ```byte```와 ```bytes1```은 동일한 의미.
24 | ```assert()```가 오류이면 ```Exception```이 발생하고, 다음 줄이 실행이 되지 않는다. 즉 ```x==1```인 경우 ```true```가 반환된다.
27~29 | ```struct```은 반환할 수 없다. 항목 하나 하나를 반납한다. **web3에서 반환할 경우에는 문자열로 변환하여서 toString() 출력**할 수 있다.
34~38 | bytes.length는 가능하다.
39~43 | a의 hex는 61이다. bytes1은 "0x61",  bytes2 "0x6161"로 입력한다.
44~48 | <string>.length는 오류.
53~57 | 한글을 입력하려면 string을 사용한다.

In [1]:
%%writefile src/DataTypeTest.sol
pragma solidity ^0.5.0;
contract DataTypeTest {
    bool isMarried=true;
    int256 x=2;
    bytes1 b1=0xFF;
    bytes23 place23="7 hongji-dong jongro-gu";
    bytes8 place8="7 hongji";
    bytes place="7 hongji-dong jongro-gu Seoul";
    string constant name="jsl";
    struct Student {
            uint num;
            string name;
            bool isEnrolled; 
    }
    Student s1=Student(201911111,"jslim",true);
    enum gender {male, female}
    function getB1() public view returns(byte) {
        return b1;
    }
    function getB23() public view returns(bytes23) {
        return place23;
    }
    function testInt() public view returns(bool) {
        assert(x==1);
        return true;
    }
    function getStudent() public view returns(uint, string memory, bool){
        return (s1.num, s1.name, s1.isEnrolled);
    }
    function getStudentName() pure public returns(string memory){
        Student memory s2=Student(201911111,"jslim",true);
        return s2.name;
    }
    function testBytes() pure public returns(uint) {
        bytes memory b="7 hongji-dong jongro-gu";
        //return place23.length;  //ok
        return b.length;        // returns 23
    }
    //need the arg in hex e.g. bytes1 "0x61" bytes2 "0x6161"
    //a 61 b 62 ... y 79
    function testBytesArg(bytes1 a) pure public returns(bytes1) {
        return a;
    }
    function testString() pure public returns(uint) {
        string memory nameLocal="jslLocal";  //should be memory
        //return nameLocal.length;  //error
        return bytes(nameLocal).length;  //casting
    }
    function testArr() pure public returns(uint) {
        uint[3] memory intArr;      // should be memory
        return intArr.length;
    }
    function getString() pure public returns(string memory) {
        string memory s="한글";
        //bytes4 memory b="한글";
        return s;
    }
}

Overwriting src/DataTypeTest.sol


### 9.4.2 address
<a id='9.4.2'></a>

address는 계정의 주소로서 20 바이트 길이를 가진다.
우리가 은행의 계좌번호를 통해 잔고를 조회하거나, 입출금을 할 수 있다.
address도 마찬가지이다.

멤버 | 설명
-----|-----
```<address>.balance (uint256)``` | 계정의 wei 잔액. 자신이 아니더라도 누구나 읽을 수 있다.
```<address>.send(uint256 amount) returns (bool)``` | 계정에서 송금 기능. 실패하면 false 반환.
```<address>.transfer()``` | 계정에서 송금 기능. 실패하면 throw 예외 발생. 따라서 send() 보다 안전
```<address>.call.value().gas()``` | 계정에서 송금기능. 실패하면 false 반환. call() 함수는 low-level 하위수gas를 처리하는 경우

다음과 같이 잔고를 조회할 수 있다. 즉 <address>.balance라고 하면 잔고를 구한다.
```python
<address>.balance
```

또한 송금을 할 수 있다.
은행거래에서도 타인의 계좌에서 임의로 인출이 허용되지 않는 것처럼
송금은 자신의 주소에서만 가능하다.
자신의 주소는 스스로 unlock, sign을 할 수 있는 권한이 있고, 그에 따라 출금이 가능한 것이 당연한다.
그러나 임의의 주소에서 누군가 출금을 한다는 것은 비상식적이다.
금액의 단위는 Wei이다.

0.5.0 버전부터 주소에서 transfer() 함수를 사용하여 송금하려면 
address payable로 선언해야 한다.

```python
address payable owner;
```

#### send()
send() 함수는 계정으로 송금을 하고, 그 결과를 **성공 또는 실패를 나타내는 boolean으로 반환**한다.
문제는 송금이 실패했을 경우에 발생한다.
송금이 성공적으로 이루어지 않았다면, ```call statck error``` 또는 ```out of gas error``` 등의 이유로 실패하는 경우에도 예외 **Exception을 발생하지 않아** 문제가 될 수 있다.
송금이 실패한 경우에 주의해야 하며, 금액을 반환하거나 적절한 처리 로직을 넣어 주어야 한다.
```!send()```라고 실패하는 경우 예외를 발생하도록 한다.

```python
if(!address.send(amount)) throw;
```

receiver.send()에 대해, receiver fallback함수가 실행되면, 최대 2300 gas를 사용할 수 있다. 2300을 초과하는 금액저장, 송금, 다른 컨트랙을 호출하면 out-of-gas 예외가 발생한다.
    
```throw```는 송금액이 원위치, 송금자에게 반환된다. gas는 당연히 소비되었으므로 송금반환되지 않는다. gas 잔액이 남아도 반환되지 않고 모두 소비된 것으로 한다. 여러 계정에 송금하는 중 throw가 발생하면, 나머지 송금계정에는 송금이 아예 발생하지 않을 수 있다.

```python
address _receiver = 0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480;
if(myAddress.balance>=10) _receiver.send(10);
```

#### transfer()
실패하면 throw 예외를 발생시키며, 호출자에게 환급된다. 따라서 send() 보다 안전하다.

```python
address.transfer(amount)
```

#### call (두 계약 사이에 call하는 경우는 다음에 설명)
* call, callcode, delegatecall은 low-level이므로 가급적 사용하지 않는다.
call함수는 gas를 정해서 처리하는 경우에 사용한다.
이 함수는 addr.send(x)와 동일한 기능을 할 수 있다. 단, 받는 측에 gas를 사용할 수 있게 하기 때문에 악의적으로 사용될 수 있다.

```python
<address>.call.value(20) //20 Wei를 송금한다.
```

**받는 측 함수를 실행하면서 송금**.
함수를 호출, gas는 2, 송금액은 20 Wei. 이 경우 gas금액이 적어서 실패하게 될 것이다.
```python
<address>.gas(2).value(20)()
```

* this는 contract 자신의 주소를 말한다.
현재 컨트랙의 잔고는 ```address(this).balance```로 구할 수 있다.
```this.balance```와 같이 하면 ```this```는 현재 인스턴스를 의미하기 때문에 잔고를 구할 수 없다.

```python
this.myFunction()
````

### 9.4.3 저장장소
<a id='9.4.3'></a>

reference type은  ```Arrays```, ```struct```, ```mapping```과 같이 32바이트 이상의 크기를 가진 복잡한 데이터 타잎이다.
이들 변수는 저장장소를 정의해 줄 수 있다.

* ```storage```는 32바이트 길이의 key-value 형식의 저장되면 계속 유지되는 경우이다. 컨트랙의 모든 함수에서 값을 볼 수 있다. 컨트랙의 상태변수는 기본 값이 ```storage```이며 일단 저장이 되면 영원히 저장된다.
* ```memory```는 함수의 지역변수. 함수 내에서만 짧은 시간 동안 사용되므로 memory 수식어를 붙여준다. 임시로 저장되는 경우에 사용한다. 지역변수와 같이 함수 내에서 국한되어 저장된다. **함수의 인자 또는 반환 값은 memory를 사용한다. 단 struct, array or mapping와 같은 reference type은 지역변수라 하더라도 storage가 기본 값이다.
* ```stack```: 지역변수 (array, struct, mapping 제외)를 저장하는 stack. 1,024 수준까지 저장할 수 있고 그 이상은 예외처리 된다. 비용은 memory와 동일하다.
* ```calldata```: **외부**에서 함수를 호출할 때 호출자가 제공하는 데이터를 저장한다.

요약하면,
* 상태변수는 storage에 저장한다.
* 지역변수 (함수인자/반환, value type 지역변수)는 memory를 기본 default로 사용한다.
단, 함수에서 사용되는 reference type (struct, array, mapping, string)은 기본이 storage이고, memory를 사용하려면 선언이 필요하다.

### 9.4.4 가시성
<a id='9.4.4'></a>

컨트랙의 state variables는 사용할 수 있는 권한을 가시성으로 제어할 수 있다.
* 가시성을 적어주지 않으면 기본 값은 ```internal```로 정의된다.
```internal```은 컨트랙 내부에서만 사용할 수 있고, 상속을 하면 자식이 물려받을 수 있다는 의미이다.
* ```external```은 ```internal```의 반대, 외부에서 호출하는 경우만 허용된다. 내부에서 사용할 경우 ```this.f()```로 호출함. 
* 외부에서 사용하려면 ```public```으로 선언하면 된다. 객체지향 ```public```과 동일한 의미이다.
* 블록체인 내부에서 컨트랙 자신만 사용할 수 있고, 객체지향 ```private```과 동일한 의미이다.

구분 | 설명 | default
-----|-----|-----
```public``` | 블럭체인 외부에서 누구나 사용 | no
```private``` |  컨트랙 자신만 사용 | no
```internal``` | 현재 컨트랙 내부에서만 또는 상속의 경우에 사용하는 경우. java의 ```protected```와 같은 의미 | yes
```external``` | 외부에서 호출하는 경우만 허용 | no


## 실습: 주소 계정에서 입출금

앞서 geth 단말에서 account[0]에서 account[1]로 전송해보았다.
컨트랙에서 송금하는 프로그램을 개발해 보자.
은행을 경유하지 않고도 송금이 이루어지는 과정을 이해해 보자.
은행없이 송금하는 경우, 서로 어떻게 신뢰할 수 있는지
실패하는 경우에는 어떻게 복구할 수 있는지 생각해 보자.


### 단계 1: 컨트랙 개발

줄 | 설명
-----|-----
5 ~ 8 | 전송자의 ```msg.value```에 입력된 ether를 contract으로 입금하는 함수이다. 주의할 점은 함수인자 amount는 ether가 아니다. 실제 입금되는 금액은 ```msg.value```이다. 단 ```msg.value```와 amount가 동일해야 입금이 실행된다. 그리고 **```payable```**로 명시해 주어야 한다.
9 ~ 11 | ```msg.sender``` 계정으로 컨트랙 자신의 잔고, ```address(this).balance```를 전부 이체한다. ```msg.sender```는 매번 메시지를 전송할 때마다 변경되는 것이 아니라, 배포하는 시점의 전송자 즉 프로그램을 배포하는 권한을 가진 사람만이 (정확히 말하면 개인키) owner가 된다.
12 ~ 14 | 컨트랙 자신의 잔고, ```address(this).balance```를 조회한다.

In [5]:
%%writefile src/myBank.sol
pragma solidity ^0.5.0;

contract myBank {
    function deposit(uint256 amount) payable public {
        //'value' of transacion object
        require(msg.value == amount);
    }
    function withdraw() public {
        msg.sender.transfer(address(this).balance);
    }
    function queryBalance() public view returns (uint256) {
        return address(this).balance;
    }
}

Overwriting src/myBank.sol


### 단계 2: 컴파일

In [6]:
!solc --gas --abi --bin src/myBank.sol


======= src/myBank.sol:myBank =======
Gas estimation:
construction:
   129 + 77200 = 77329
external:
   deposit(uint256):	259
   queryBalance():	601
   withdraw():	infinite
Binary: 
608060405234801561001057600080fd5b50610182806100206000396000f3fe608060405260043610610051576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c61146100565780633ccfd60b14610081578063b6b55f2514610098575b600080fd5b34801561006257600080fd5b5061006b6100c6565b6040518082815260200191505060405180910390f35b34801561008d57600080fd5b506100966100e5565b005b6100c4600480360360208110156100ae57600080fd5b8101908080359060200190929190505050610145565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b3373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f19350505050158015610142573d6000803e3d6000fd5b50565b803414151561015357600080fd5b5056fea165627a7a723058200be23183b595a23b19c1edd28a441bd6edb3c079b72867b70

### 단계 3: 컨트랙 배포

In [8]:
%%writefile src/myBankDeploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"}];
var _bin="608060405234801561001057600080fd5b50610182806100206000396000f3fe608060405260043610610051576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c61146100565780633ccfd60b14610081578063b6b55f2514610098575b600080fd5b34801561006257600080fd5b5061006b6100c6565b6040518082815260200191505060405180910390f35b34801561008d57600080fd5b506100966100e5565b005b6100c4600480360360208110156100ae57600080fd5b8101908080359060200190929190505050610145565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b3373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f19350505050158015610142573d6000803e3d6000fd5b50565b803414151561015357600080fd5b5056fea165627a7a723058200be23183b595a23b19c1edd28a441bd6edb3c079b72867b70c48245fbfc5bddb0029";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new("hello world",{data:"0x"+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

Writing src/myBankDeploy.js


In [9]:
!node src/myBankDeploy.js

contractAddress:  undefined
transactionHash:  0x169bdad38e4fd3535661a206d1fd8853709daef844ac5e7f9bbdb5e5f289bc0b
contractAddress:  0x646570db59e036e51a881f5e46722216ccbe7f52
transactionHash:  0x169bdad38e4fd3535661a206d1fd8853709daef844ac5e7f9bbdb5e5f289bc0b


### 단계 4: 사용

```deposit()```함수의 ```value:1111``` 필드를 채워주면 ```msg.value```로 전달이 된다.
이 때 함수의 인자도 동일하게 1111 Wei를 넣어준다.

```python
myBank.deposit(1111,{from:web3.eth.accounts[0],gas:80000,value:1111})
```

마이닝을 하고나면 잔고가 1111이 된다.
컨트랙의 ```deposit()``` 함수에서 잔고에 합산을 하지 않아도 된다.

자신의 잔고에서 1111만큼 빠져나간 것을 확인해보자.
```python
> eth.getBalance(eth.accounts[0]);
94999999999999998889
```

In [9]:
%%writefile src/myBankUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"withdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"}];
var _contract = web3.eth.contract(_abiArray);
var myBank = _contract.at("0x646570db59e036e51a881f5e46722216ccbe7f52");
console.log(web3.eth.getBalance(web3.eth.accounts[0]).toNumber());
console.log(myBank.deposit(1111,{from:web3.eth.accounts[0],gas:80000,value:1111}));
var bal=myBank.queryBalance.call();
console.log(bal.toString());
console.log(web3.eth.getBalance(web3.eth.accounts[0]).toNumber());
myBank.withdraw.sendTransaction({from:web3.eth.accounts[0]});

Overwriting src/myBankUse.js


In [10]:
!node src/myBankUse.js

755000000000000000000
0x423d8029c8b70009da4001184ba989475225ba2d8d8cdb5cb9a7b5e5ece207cf
4444
755000000000000000000


1111을 deposit() 입금하면 잔고에 반영이 되는 것을 볼 수 있다.
또한 withdraw() 출금하면 잔고가 0이 되면서 전송자의 잔고가 증가한다.

In [11]:
!node src/myBankUse.js

760000000000000000000
0x4026b7152afdffc69b56c5799e7bce73ce886687a7f0a4e313018c84a772a1d3
0
760000000000000000000


### 9.4.5 conversion
<a id='9.4.5'></a>

형변환을 할 경우에는 데이터 타잎의 크기에 주의해야 한다.
예를 들어 int8에서 int로 변환할 경우 괄호 안에 써주면 된다.
이 경우 int는 256비트이므로 오류없이 변환이 된다.
```python
int8 i8=20;
uint i256=int(i8);
```

uint는 bytes32로 변환할 수 있다.
```python
uint x=20;
bytes32(x);
```

int를 bytes로 바로 형변환을 할 수 없다.
int는 int256, 즉 32바이트라서 bytes32로 변환하고, 이를 bytes로 변환해야 한다.
이 경우 바이트마다 옮겨주어야 한다.

```0x68656c6c```은 16진수, 8자리이다.
한 자리에는 0,1,2...D,E,F까지 $2^4$ 가지를 표현할 수 있고, 4비트가 소요된다.
이 데이터 bytes4를 bytes2로 변환해보자.
**오버플로우는 위 4자리부터 잘려 나가게 된다**.
따라서 ```0x6865```가 저장된다.

```python
bytes4 b4 = 0x68656c6c;
bytes2(b4);
```

```string```을 ```bytes```로 변환하는 경우:

```python
string s = "hello";
bytes b = bytes(s); //0x68656c6c6f
```

hex | char
-----|-----
68 | h
65 | e
6c | l
6c | l
6f | o

In [2]:
%%writefile src/DataConversionTest.sol
pragma solidity ^0.5.0;
contract DataConversionTest {
    function convertInt8ToInt() view public returns(int) {
        int8 i8=20;
        return int(i8);
    }
    function convertIntToBytes32() view public returns(bytes32) {
        uint x=20;
        return bytes32(x);
    }
    function convertInt64ToBytes8() view public returns(bytes8) {
        uint64 x=30;
        return bytes8(x);
    }
    function convertIntToBytes() view public returns(bytes memory) {
        // can not convert uint -> bytes
        // convert uint -> bytes32 -> bytes
        uint x = 20;
        bytes32 y = bytes32(x); //uint=uint256
        // can not convert from bytes32 -> bytes;
        bytes memory z = new bytes(32);
        for (uint i=0; i < 32; i++) {
            z[i] = y[i];
        }
        return z;
    }
    function convertStringToBytes() view public returns(bytes memory) {
        string memory s="hello";
        return bytes(s);  //0x68656c6c6f
    }
    function convertBytes4ToBytes2() view public returns(bytes2) {
        bytes4 b4=0x68656c6c;
        return bytes2(b4);
    }
    function convertBytes4ToInt32() view public returns(int32) {
        bytes4 b4=0x68656c6c;
        return int32(b4);
    }
}

Writing src/DataConversionTest.sol


### 9.4.6 전역 변수
<a id='9.4.6'></a>

* 화폐 단위 ```wei``` $10^{18}$로 표현되며, 1 ```ether```는 ```1000000000000000000```이다.

```python
1 ether == 1000 finney
```

* 시간 ```years```는 지원되지 않으며, ```seconds```로 표시된다. ```now```는 현재 시간, ```block.timestamp```와 동일하다.
```python
1 == 1 seconds, 1 days == 86400 seconds
```

* tx 컨트랙을 호출하는 트랜잭션 관련 정보
    * ```tx.origin``` 트랜잭션에 사인한 계정
    * ```tx.gasprice``` 트랜잭션 호출자가 명시한 gas price

* msg 컨트랙의 함수를 호출한 전송 관련 정보
    * ```msg.data``` | call 데이터 (bytes 값으로 표현)
    * ```msg.gas``` | gas 잔여분
    * ```msg.sender``` | 현재 함수를 호출하는 측의 주소
        * 거래가 호출자U1 -> 컨트랙C1 -> 컨트랙C2의 순서대로 완성이 될 경우:
        * C2에서는 ```msg.sender```는 바로 직전의 호출자C1를 말하며, 따라서 컨트랙도 가능하다.
        * 반면에 ```tx.origin```는 컨트랙이 될 수 없다.
    * ```msg.value``` | 컨트랙에 지급되는 ether (단위는 wei)

* block
    * ```block.coinbase``` | 현재 블록 마이너의 주소
    * ```block.difficulty``` | 현재 블록의 난이도
    * ```block.gaslimit``` | 현재 블록의 gaslimit
    * ```block.number``` | 현재 블록 수
    * ```block.blockhash``` | 현재 블록 해쉬 값
    * ```block.timestamp``` | 현재 블록 타임스탬프 epoch (1970년 1월 1일 0시) 이후 지나간 초, uint256.

In [4]:
%%writefile src/GlobalVarsTest.sol
pragma solidity ^0.5.0;
contract GlobalVarsTest {
    function getEther() view public returns(uint) {
        return 1 ether;  //1000000000000000000
    }
    function getDays() view public returns(uint,uint) {
        require(block.timestamp==now);
        return (now, 1 day); // 1558816133 86400 ??web3? 1 days?
    }
    function getMsgValue() public payable returns(uint) {
        return msg.value;
    }
    function getMsgSender() view public returns(address) {
        return msg.sender;
    }
    function getCoinbase() view public returns(address) {
        return block.coinbase;
    }
    function getBlockNumber() view public returns(uint) {
        return block.number;
    }
    function getBlockTimeStamp() view public returns(uint) {
        return block.timestamp;
    }
}

Overwriting src/GlobalVarsTest.sol


## 실습: 은행 송금

앞서 컨트랙을 수정해서 다른 계정으로 송금할 수 있도록 해보자.

### 단계 1: 컨트랙 개발

줄 | 설명
-----|-----
4 | ```address```선언
5 ~ 7 | 생성자에 ```msg.sender```를 ```owner```로 설정
8 | ```event``` 설정
10 ~ 14 | 송금. ```owner```만 송금할 수 있고 ```event``` 발생
21 ~ 24 | ```owner```에게 전액 출금

In [12]:
%%writefile src/BankV2.sol
pragma solidity ^0.5.0;

contract BankV2 {
    address owner;
    constructor() public {
        owner=msg.sender;
    }
    event Sent(address from, address to, uint amount );
    //function send(address payable _receiver, uint _amount) public payable {
    function sendTo(address payable _receiver) public payable {
        require(msg.sender==owner);
        _receiver.transfer(msg.value);
        emit Sent(msg.sender, _receiver, msg.value);
    }
    function queryBalance() public view returns(uint) {
        return address(this).balance;
    }
    function deposit(uint amount) public payable {
        require(msg.value==amount);
    }
    function widthdraw() public {
        require(msg.sender==owner);
        msg.sender.transfer(address(this).balance);
    }
}

Overwriting src/BankV2.sol


### 단계 2: 컴파일

In [13]:
!solc --gas --abi --bin src/BankV2.sol


======= src/BankV2.sol:BankV2 =======
Gas estimation:
construction:
   20486 + 176000 = 196486
external:
   deposit(uint256):	259
   queryBalance():	601
   sendTo(address):	infinite
   widthdraw():	infinite
Binary: 
608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610370806100606000396000f3fe60806040526004361061005c576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c611461006157806352b50a2a1461008c578063b6b55f25146100a3578063e6d25245146100d1575b600080fd5b34801561006d57600080fd5b50610076610115565b6040518082815260200191505060405180910390f35b34801561009857600080fd5b506100a1610134565b005b6100cf600480360360208110156100b957600080fd5b81019080803590602001909291905050506101ef565b005b610113600480360360208110156100e757600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610200565b005b60003073fffffffffffffffffffffffffff

### 단계 3: 컨트랙 배포

In [14]:
%%writefile src/BankV2Deploy.js
var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
}
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _bin="608060405234801561001057600080fd5b50336000806101000a81548173ffffffffffffffffffffffffffffffffffffffff021916908373ffffffffffffffffffffffffffffffffffffffff160217905550610370806100606000396000f3fe60806040526004361061005c576000357c01000000000000000000000000000000000000000000000000000000009004806336f40c611461006157806352b50a2a1461008c578063b6b55f25146100a3578063e6d25245146100d1575b600080fd5b34801561006d57600080fd5b50610076610115565b6040518082815260200191505060405180910390f35b34801561009857600080fd5b506100a1610134565b005b6100cf600480360360208110156100b957600080fd5b81019080803590602001909291905050506101ef565b005b610113600480360360208110156100e757600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff169060200190929190505050610200565b005b60003073ffffffffffffffffffffffffffffffffffffffff1631905090565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561018f57600080fd5b3373ffffffffffffffffffffffffffffffffffffffff166108fc3073ffffffffffffffffffffffffffffffffffffffff16319081150290604051600060405180830381858888f193505050501580156101ec573d6000803e3d6000fd5b50565b80341415156101fd57600080fd5b50565b6000809054906101000a900473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff163373ffffffffffffffffffffffffffffffffffffffff1614151561025b57600080fd5b8073ffffffffffffffffffffffffffffffffffffffff166108fc349081150290604051600060405180830381858888f193505050501580156102a1573d6000803e3d6000fd5b507f3990db2d31862302a685e8086b5755072a6e2b5b780af1ee81ece35ee3cd3345338234604051808473ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff1681526020018373ffffffffffffffffffffffffffffffffffffffff1673ffffffffffffffffffffffffffffffffffffffff168152602001828152602001935050505060405180910390a15056fea165627a7a723058202e512a1df8dbf7513c14646ff33b94f8e9ee902911138575219f157ffdc92ddc0029";
var _contract = web3.eth.contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
var _instance=_contract.new({data:"0x"+_bin,from:web3.eth.accounts[0],gas:1000000}, function(err, contract) {
    if (!err) {
        console.log("contractAddress: ", contract.address);
        console.log("transactionHash: ", contract.transactionHash);
    }
});

Writing src/BankV2Deploy.js


In [15]:
!node src/BankV2Deploy.js

contractAddress:  undefined
transactionHash:  0x9d41a3355b7ae971cbadcd92601f0ec6596678feec93ad1f2f0ae85b34e5e064
contractAddress:  0xeed4d219cc42d511d17e65fbd3d1b880f62132a8
transactionHash:  0x9d41a3355b7ae971cbadcd92601f0ec6596678feec93ad1f2f0ae85b34e5e064


### 단계 4: 사용

아래에서는 계정의 주소를 **따옴표 없이 사용**하면 값이 변경되는 것을 보여주고 있다. args출력을 비교해 보면 원래 함수 인자에서 주어진 값과 다르게 변경된 것을 알 수 있다
```python
args: 
   { from: '0x21c704354d07f804bab01894e8b4eb4e0eba7451',
     to: '0x778ea91cb0d08927fa4bf3f90a7ccbb700000000',
          // 원래 함수 인자와 다름 0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480
     amount: { [String: '555'] s: 1, e: 2, c: [Object] } }
```

In [16]:
%%writefile src/BankV2Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var bank = _contract.at("0xeed4d219cc42d511d17e65fbd3d1b880f62132a8");
var filter = bank.Sent(function (error, result) {
  if (!error)
    console.log(result);
});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));
console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, {from:web3.eth.accounts[0],gas:100000, value:555}));

Writing src/BankV2Use.js


In [ ]:
!node src/BankV2Use.js

0x6a693f3558f73c8472a84743a5f419f8d9506716ca2aa2a008d7259ef94079c4
{ address: '0xeed4d219cc42d511d17e65fbd3d1b880f62132a8',
  blockNumber: 263,
  transactionHash: '0x6a693f3558f73c8472a84743a5f419f8d9506716ca2aa2a008d7259ef94079c4',
  transactionIndex: 0,
  blockHash: '0xc83c822d9c9f259aaa746a4f1b871c03352bbf69e649a75fb8591ad73745866a',
  logIndex: 0,
  removed: false,
  event: 'Sent',
  args: 
   { from: '0x21c704354d07f804bab01894e8b4eb4e0eba7451',
     to: '0x778ea91cb0d08927fa4bf3f90a7ccbb700000000',
     amount: { [String: '555'] s: 1, e: 2, c: [Object] } } }


In [ ]:
%%writefile src/BankV2Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":true,"inputs":[],"name":"queryBalance","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"},{"constant":false,"inputs":[],"name":"widthdraw","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[{"name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"sendTo","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var bank = _contract.at("0xeed4d219cc42d511d17e65fbd3d1b880f62132a8");
var filter = bank.Sent(function (error, result) {
  if (!error)
    console.log(result);
});
//console.log(bank.sendTo(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));
console.log(bank.sendTo("0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480", {from:web3.eth.accounts[0],gas:100000, value:555}));

In [ ]:
!node src/BankV2Use.js

0xf7d137adac6cae468b71e9b06741cfec8371f557ef46ce9645dd5c083012237e
{ address: '0xeed4d219cc42d511d17e65fbd3d1b880f62132a8',
  blockNumber: 45482,
  transactionHash: '0xf7d137adac6cae468b71e9b06741cfec8371f557ef46ce9645dd5c083012237e',
  transactionIndex: 0,
  blockHash: '0x407ba95c4153a62c358dbed72fae83de025b8bd1071a51e20551e7f767824b43',
  logIndex: 0,
  removed: false,
  event: 'Sent',
  args: 
   { from: '0x21c704354d07f804bab01894e8b4eb4e0eba7451',
     to: '0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480',
     amount: { [String: '555'] s: 1, e: 2, c: [Object] } } }

받는 주소를 account[1]로 했더니 올바르게 
```python
> eth.getBalance(eth.accounts[1]);
0   <--- 이전 잔고
> eth.pendingTransactions
[{
    blockHash: null,
    blockNumber: null,
    from: "0x21c704354d07f804bab01894e8b4eb4e0eba7451",
    gas: 100000,
    gasPrice: 500000000000,
    hash: "0xa4c9e98767fbe60a0722b3d31647ca351d56ef7a960f00d20edbc6e26a98c94f",
    input: "0x3e58c58c000000000000000000000000778ea91cb0d0879c22ca20c5aea6fbf8cbeed480",
    nonce: 32,
    r: "0x733d32c710bc4d4944da811daa0d639b3a0241cd3994039600c73d69f247a6d",
    s: "0x78476837a95b30c7991af36c32eccbe4ee4f375349adb012db3c8f0c919a0213",
    to: "0x53fd2f60c463f19c8c7bcdb5e699cac94b8300b2",
    transactionIndex: 0,
    v: "0x66",
    value: 555 <--- 전송될 금액
}]
> miner.start(1);admin.sleepBlocks(1);miner.stop();
null
> eth.getBalance(eth.accounts[1]);
555 <--- 이후 잔고
```

In [ ]:
%%writefile src/BankV2Use.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://117.16.44.45:8445"));
var _abiArray=[{"constant":false,"inputs":[{"name":"_receiver","type":"address"}],"name":"send","outputs":[],"payable":true,"stateMutability":"payable","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"name":"from","type":"address"},{"indexed":false,"name":"to","type":"address"},{"indexed":false,"name":"amount","type":"uint256"}],"name":"Sent","type":"event"}];
var _contract = web3.eth.contract(_abiArray);
var sendTest = _contract.at("0x53fd2f60c463f19c8c7bcdb5e699cac94b8300b2");
var filter = sendTest.Sent(function (error, result) {
  if (!error)
    console.log(result);
});
//console.log(sendTest.send(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, 100,{from:web3.eth.accounts[0],gas:100000}));
//console.log(sendTest.send(0x778ea91cb0d0879c22ca20c5aea6fbf8cbeed480, {from:web3.eth.accounts[0],gas:100000, value:555}));
console.log(sendTest.send(web3.eth.accounts[1], {from:web3.eth.accounts[0],gas:100000, value:555}));

In [1]:
!node src/BankV2Use.js

Overwriting src/SendTest.sol
